# Random Species Generator

In [1]:
import pandas as pd
import geopandas as gpd
import json
from ast import literal_eval
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import random
#pip install geopandas
#pip install spacy
#python -m spacy download en_core_web_sm

In [5]:
# read in the dataset 
df = pd.read_json('animal_news_database.json', lines = True)

# Read in the datasets defining our subset data
undoc = pd.read_csv("undoc_species.csv", sep = ";")
charismatic = pd.read_csv("charismatic_20.csv", sep = ";")
edge = pd.read_csv("non_charismatic_20.csv", sep = ";")



# Clean up datasets
charismatic = charismatic.rename(columns = {'Scientifc Name' : 'scientific_name'})
charismatic = charismatic[['scientific_name', 'Ecosystem', 'Class']]
edge = edge.rename(columns = {'Scientifc Name' : 'scientific_name'})
edge = edge[['scientific_name', 'Ecosystem', 'Conservation Attention', 'Class']]

# Drop Twitter handles as they are not species names
df = df[df.scientific_name.str.contains('@') == False]
df.reset_index(drop = True, inplace = True)
# How many species listed total in this dataset
#df['scientific_name'].nunique()

In [6]:
charismatic

,scientific_name,Ecosystem,Class
0,Megaptera novaeangliae,Aquatic,Mammal
1,Panthera leo,Terrestrial,Mammal
2,Panthera onca,Terrestrial,Mammal
3,Panthera pardus,Terrestrial,Mammal
4,Ailuropoda melanoleuca,Terrestrial,Mammal
5,Acinonyx jubatus,Terrestrial,Mammal
6,Canis lupus,Terrestrial,Mammal
7,Gorilla beringei,Terrestrial,Mammal
8,Equus grevyi,Terrestrial,Mammal
9,Crocodylus porosus,Aquatic,Reptile


In [26]:
edge.query('Ecosystem == "Aquatic"').sample(n=8)

,scientific_name,Ecosystem,Conservation Attention,Class
19,Monachus monachus,Aquatic,Medium,Mammalia
1,Andrias davidianus,Aquatic,Good,Amphibia
8,Gavialis gangeticus,Aquatic,Good,Reptilia
7,Eretmochelys imbricata,Aquatic,Good,Reptilia
11,Platysternon megacephalum,Aquatic,Low,Reptilia
2,Pristis zijsron,Aquatic,Low,Chondrichthyes
0,Pristis pristis,Aquatic,Good,Chondrichthyes
13,Crocodylus cataphractus / Mecistops cataphractus,Aquatic,Medium,Reptilia


In [22]:
# Charismatic species
species_class = ["Mammal"] * 8 + ["Reptile"] * 2
ecosystem = ["Terrestrial"] * 6 + ["Aquatic"] * 4
random.shuffle(ecosystem)  # randomly shuffle ecosystem
random.shuffle(species_class)

df1_selected = pd.DataFrame(columns=charismatic.columns)
for i in range(10):
    df1_selected = df1_selected.append(
        charismatic.query(f"`Class` == {species_class}[{i}] and Ecosystem == {ecosystem}[{i}]").sample(n=1)
    )

/tmp/ipykernel_94/3109963767.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1_selected = df1_selected.append(
/tmp/ipykernel_94/3109963767.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1_selected = df1_selected.append(
/tmp/ipykernel_94/3109963767.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1_selected = df1_selected.append(
/tmp/ipykernel_94/3109963767.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1_selected = df1_selected.append(
/tmp/ipykernel_94/3109963767.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1_se

In [30]:
# Charismatic species
species_class2 = ["Mammalia"] * 4 + ["Reptilia"] * 3 + ["Aves"] * 2 + ["Chondrichthyes"] * 1
ecosystem2 = ["Terrestrial"] * 6 + ["Aquatic"] * 4
random.shuffle(ecosystem2)  # randomly shuffle ecosystem
random.shuffle(species_class2)

df2_selected = pd.DataFrame(columns=edge.columns)
for i in range(10):
    df2_selected = df2_selected.append(
        edge.query(f"`Class` == {species_class2}[{i}] and Ecosystem == {ecosystem2}[{i}]").sample(n=1)
    )
    
df2_selected.nunique()

/tmp/ipykernel_94/3020573803.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2_selected = df2_selected.append(
/tmp/ipykernel_94/3020573803.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2_selected = df2_selected.append(
/tmp/ipykernel_94/3020573803.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2_selected = df2_selected.append(


ValueError: a must be greater than 0 unless no samples are taken

In [ ]:
# Add new column for storing the values article amount found under each species type 
for i, row in df.iterrows():
    df.at[i, 'article_amt'] = len(row.articles.values())
    
# Read in the CITES data to join with the top 10 (or 20) most reported on species 
cites1 = pd.read_csv("cites1.csv", delimiter = ';')

# Keep only most relevant columns
cites1 = cites1[['Kingdom', 'Class', 'Order', 'Family', 'Genus', 'Species', 'Scientific Name', 'Listing', 'NativeDistributionFullNames', 'Extinct_Distribution']]
cites1 = cites1.rename(columns = {'Scientific Name' : 'scientific_name'})

# Do the same for the entire dataframe
df = df.merge(cites1, on = 'scientific_name', how = 'inner')

# Merge similarly with the data subsets
rizz = df.merge(charismatic, on = 'scientific_name', how = 'inner')
dg = df.merge(edge, on = 'scientific_name', how = 'inner')

# Group dataframe by the scientific name that remains consistent for all rows 
grouped = rizz.groupby('scientific_name')# 'Class', 'iucn_status'])
# Calculate the totals of the group species
article_totals_rz = grouped.article_amt.sum().round()

# Group dataframe by the scientific name that remains consistent for all rows 
grouped = dg.groupby('scientific_name')# 'Class', 'iucn_status'])
# Calculate the totals of the group species
article_totals_dg = grouped.article_amt.sum().round()

In [ ]:
# This will at least return unique instances. Does not include the Ecosystem requirements I came up with 
rizz.query("Class = Mammal").sample(n=8)

rizz.query("Class = Reptile").sample(n=2)

In [ ]:
species_class = ["Mammalia"] * 8 + ["Reptilia"] * 2
ecosystem = ["Terrestrial"] * 6 + ["Aquatic"] * 4
random.shuffle(ecosystem)  # randomly shuffle ecosystem
random.shuffle(species_class)


# POssible solution... results in an Variable errore and doesn't select unique species..
df1_selected = pd.DataFrame(columns=df1.columns)
for i in range(10):
    df1_selected = df1_selected.append(
        df1.query(f"Class = {species_class[i]} and Ecosystem = {ecosystem[i]}").sample(n=1)
    )
    
rizz_selected = pd.DataFrame(columns=rizz.columns)
for i, j in zip(species_class, ecosystem):
    rizz_selected = rizz_selected.append(
        rizz.query(f'`Class` == @i and Ecosystem == @j').sample(n=1)
    )
    


In [ ]:
from random import sample
sample([(charismatic.loc[charismatic['Class'] == 'Mammalia']), (charismatic.loc[charismatic['Class'] == 'Reptilia'])], counts=[8, 2], k=10) 
